# Import

In [1]:
import pandas as pd

df_drug = pd.read_csv('dataset/train.csv',sep=',')

In [2]:
df_drug

,patient_id,name_of_drug,use_case_for_drug,review_by_patient,effectiveness_rating,drug_approved_by_UIC,number_of_times_prescribed,base_score
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,20-May-12,27,8.022969
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,27-Apr-10,192,7.858458
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,14-Dec-09,17,6.341969
3,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9,27-Nov-16,37,6.590176
4,155963,Cialis,Benign Prostatic Hyperplasia,"""2nd day on 5mg started to work with rock hard...",2,28-Nov-15,43,6.144782
...,...,...,...,...,...,...,...,...
32160,183202,Cymbalta,Anxiety,"""I have been taking Cymbalta for 15 months now...",9,10-Jun-13,89,6.963020
32161,109111,Nexplanon,Birth Control,"""I have had the Nexplanon since Dec. 27, 2016 ...",6,6-Apr-17,0,0.899076
32162,121154,Venlafaxine,Panic Disorde,"""Had panic attacks and social anxiety starting...",9,10-Nov-16,25,6.241812
32163,45410,Fluoxetine,Obsessive Compulsive Disorde,"""I have been off Prozac for about 4 weeks now....",8,21-Jan-15,22,7.940428


In [3]:
df_drug.columns

Index(['patient_id', 'name_of_drug', 'use_case_for_drug', 'review_by_patient',
       'effectiveness_rating', 'drug_approved_by_UIC',
       'number_of_times_prescribed', 'base_score'],
      dtype='object')

In [4]:
df_drug.drop(['drug_approved_by_UIC','number_of_times_prescribed','base_score','effectiveness_rating','patient_id'],axis=1)

,name_of_drug,use_case_for_drug,review_by_patient
0,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati..."
1,Guanfacine,ADHD,"""My son is halfway through his fourth week of ..."
2,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh..."
3,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around..."
4,Cialis,Benign Prostatic Hyperplasia,"""2nd day on 5mg started to work with rock hard..."
...,...,...,...
32160,Cymbalta,Anxiety,"""I have been taking Cymbalta for 15 months now..."
32161,Nexplanon,Birth Control,"""I have had the Nexplanon since Dec. 27, 2016 ..."
32162,Venlafaxine,Panic Disorde,"""Had panic attacks and social anxiety starting..."
32163,Fluoxetine,Obsessive Compulsive Disorde,"""I have been off Prozac for about 4 weeks now...."


In [5]:
# Combine the columns into a single text column called 'combined_text'
df_drug['combined_text'] = df_drug.apply(
    lambda x: f"Drug: {x['name_of_drug']} | Use Case: {x['use_case_for_drug']} | Review: {x['review_by_patient']}",
    axis=1
)


In [11]:
# Texte avec le nombre minimum de caractères
min_text = df_drug.loc[df_drug['combined_text'].apply(len).idxmin(), 'combined_text']

# Texte avec le nombre maximum de caractères
max_text = df_drug.loc[df_drug['combined_text'].apply(len).idxmax(), 'combined_text']

# Afficher les longueurs
min_len = len(min_text)
max_len = len(max_text)

print(f"Texte avec le moins de caractères : {min_text} ({min_len} caractères)")
print(f"Texte avec le plus de caractères : {max_text} ({max_len} caractères)")


Texte avec le moins de caractères : Drug: Vitapap | Use Case: Pain | Review: "Good" (47 caractères)
Texte avec le plus de caractères : Drug: Acetaminophen / aspirin / caffeine | Use Case: Migraine | Review: "Like all medications I&#039;ve ever taken, I followed the directions exactly (&quot;take 2 caplets with a glass of water&quot;) and the product worked great, with no side effects. With regard to those complaining that the product was ineffective or gave you bad side effects, PLEASE read the following. Although Excedrin Migraine and Excedrin Extra Strength are the exact same product, it is vitally important to follow the Excedrin Migraine directions if you are taking the pills for a migraine, NOT the directions for Extra Strength Excedrin! Here&#039;s why. Many people have incorrectly assumed that because the two products are exactly the same, which they indeed are, that the directions are also the same. THIS IS COMPLETELY FALSE. The reason that there is a separate product solely fo

In [12]:
# Compter le nombre de textes avec plus de 2000 caractères
count_above_2000 = df_drug['combined_text'].apply(len).gt(2000).sum()

print(f"Nombre de textes avec plus de 2000 caractères : {count_above_2000}")


Nombre de textes avec plus de 2000 caractères : 10


In [14]:
from transformers import AutoTokenizer

# Modèles et leurs limites
models = {
    "multi-qa-mpnet-base-dot-v1": {"tokenizer": AutoTokenizer.from_pretrained("sentence-transformers/multi-qa-mpnet-base-dot-v1"), "max_tokens": 512},
    "all-MiniLM-L6-v2": {"tokenizer": AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2"), "max_tokens": 384},
    "all-MiniLM-L12-v1": {"tokenizer": AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L12-v1"), "max_tokens": 384},
}

# Calculer combien de textes dépassent la limite pour chaque modèle
exceeds_limit = {}
for model_name, data in models.items():
    tokenizer = data["tokenizer"]
    max_tokens = data["max_tokens"]
    
    # Calculer le nombre de tokens pour chaque texte
    df_drug[f"{model_name}_token_count"] = df_drug['combined_text'].apply(lambda x: len(tokenizer.tokenize(x)))
    
    # Vérifier les textes qui dépassent la limite
    exceeds_limit[model_name] = (df_drug[f"{model_name}_token_count"] > max_tokens).sum()

# Afficher les résultats
for model_name, count in exceeds_limit.items():
    print(f"Nombre de textes qui dépassent la limite pour {model_name}: {count}")


Token indices sequence length is longer than the specified maximum sequence length for this model (541 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (541 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (541 > 512). Running this sequence through the model will result in indexing errors


Nombre de textes qui dépassent la limite pour multi-qa-mpnet-base-dot-v1: 9
Nombre de textes qui dépassent la limite pour all-MiniLM-L6-v2: 27
Nombre de textes qui dépassent la limite pour all-MiniLM-L12-v1: 27


In [7]:
len(max(df_drug['combined_text']))

657

In [6]:
df_drug

,patient_id,name_of_drug,use_case_for_drug,review_by_patient,effectiveness_rating,drug_approved_by_UIC,number_of_times_prescribed,base_score,combined_text
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,20-May-12,27,8.022969,Drug: Valsartan | Use Case: Left Ventricular D...
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,27-Apr-10,192,7.858458,"Drug: Guanfacine | Use Case: ADHD | Review: ""M..."
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,14-Dec-09,17,6.341969,Drug: Lybrel | Use Case: Birth Control | Revie...
3,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9,27-Nov-16,37,6.590176,Drug: Buprenorphine / naloxone | Use Case: Opi...
4,155963,Cialis,Benign Prostatic Hyperplasia,"""2nd day on 5mg started to work with rock hard...",2,28-Nov-15,43,6.144782,Drug: Cialis | Use Case: Benign Prostatic Hype...
...,...,...,...,...,...,...,...,...,...
32160,183202,Cymbalta,Anxiety,"""I have been taking Cymbalta for 15 months now...",9,10-Jun-13,89,6.963020,"Drug: Cymbalta | Use Case: Anxiety | Review: ""..."
32161,109111,Nexplanon,Birth Control,"""I have had the Nexplanon since Dec. 27, 2016 ...",6,6-Apr-17,0,0.899076,Drug: Nexplanon | Use Case: Birth Control | Re...
32162,121154,Venlafaxine,Panic Disorde,"""Had panic attacks and social anxiety starting...",9,10-Nov-16,25,6.241812,Drug: Venlafaxine | Use Case: Panic Disorde | ...
32163,45410,Fluoxetine,Obsessive Compulsive Disorde,"""I have been off Prozac for about 4 weeks now....",8,21-Jan-15,22,7.940428,Drug: Fluoxetine | Use Case: Obsessive Compuls...


# Automatisation

In [ ]:
import os
import json
import google.generativeai as genai
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import torch
import time
import random 
import os

# Lire la clé API depuis API.txt
with open("API.txt", "r") as file:
    api_key = file.read().strip()

# Configurer l'API Gemini avec la clé
genai.configure(api_key=api_key)

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Initialisez l'instance de ChromaDB (Vector Store)

db_path = "chroma_db"
if not os.path.exists(db_path):
    os.makedirs(db_path)

chroma_client = chromadb.PersistentClient(path=db_path)
collection = chroma_client.get_or_create_collection(name="drug_embeddings")

# Charger le modèle d'embedding
embedding_model_name = 'sentence-transformers/all-MiniLM-L6-v2'
embedding_model = SentenceTransformer(embedding_model_name)
embedding_model.to("cuda" if torch.cuda.is_available() else "cpu")

def add_embeddings_to_vectorstore(df):
    # Vérifiez si la collection est vide en utilisant une autre méthode
    if collection.count() == 0:  # Vérifie le nombre total de documents dans la collection
        for index, row in df.iterrows():
            embedding = embedding_model.encode(row['combined_text'], device=device).tolist()
            collection.add(
                documents=[row['combined_text']],
                embeddings=[embedding],
                ids=[str(index)]
            )
        print("Embeddings ajoutés à ChromaDB.")
    else:
        print("Les embeddings existent déjà dans ChromaDB.")

# Ajoutez vos données existantes
add_embeddings_to_vectorstore(df_drug)

# Fonction pour utiliser le modèle Gemini
def query_gemini_with_retry(prompt, model_name="gemini-1.5-flash", retries=1):
    for attempt in range(retries):
        try:
            model = genai.GenerativeModel(model_name)
            response = model.generate_content(prompt)
            return response.text.strip()
        except Exception as e:
            print(f"Attempt {attempt + 1} failed: {e}")
            if attempt < retries - 1:
                time.sleep(2 ** attempt + random.random())  # Exponential backoff
            else:
                raise

# Fonction RAG pour répondre aux questions
def rag_pipeline(query, results_number=10, llm_model_name="gemini-1.5-flash"):
    # Générer l'embedding de la requête
    query_embedding = embedding_model.encode(query).tolist()
    
    # Rechercher les contextes pertinents dans ChromaDB
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=results_number
    )
    
    # Construire le contexte pour le modèle LLM
    contexts = results["documents"][0]
    context_text = "\n".join([f"Context {i + 1}: {text}" for i, text in enumerate(contexts)])
    input_prompt = f"""
It's a school project. You are an AI assistant tasked with answering questions using only the information in the provided context. Do not add any extra information or assumptions.

Context:
{context_text}

Question:
{query}

Instructions:
1. Use only the information in the context to answer the question.
2. If the context mentions multiple options, provide a list of those options clearly.
3. If the context does not provide relevant information, state: "The context does not contain enough information to answer this question."
4. Do not include any policy or ethical reasoning in your response.
5. Dont quote the Context and dont mention that you used a context/reviews to answer

Answer:
"""
    # Générer une réponse à l'aide de Gemini
    response = query_gemini_with_retry(input_prompt, model_name=llm_model_name)
    return response

# Charger les questions depuis le fichier JSON
with open("questions.json", "r") as file:
    questions_data = json.load(file)

# Stocker les résultats
results = {
    "meta": {
        "embedding_model": embedding_model_name,
        "llm_model": "gemini-1.5-flash",
        "results_number": 10
    },
    "qa_pairs": []
}

# Limiter à 10 premières questions
questions_subset = questions_data

# Itérer sur les 10 premières questions et collecter les réponses
for question_obj in questions_subset:
    question = question_obj["question"]
    answer = rag_pipeline(question)
    results["qa_pairs"].append({"question": question, "answer": answer})
    time.sleep(10)  # Petite pause entre les appels pour éviter les dépassements de quota

# Définir le répertoire ou fichier cible pour sauvegarder les résultats
output_directory = "comparaisonLLMemb"
os.makedirs(output_directory, exist_ok=True)  # Crée le répertoire s'il n'existe pas

# Générer un nom de fichier dynamique
output_filename = os.path.join(
    output_directory, 
    f"results_{embedding_model_name.split('/')[-1]}_{results['meta']['llm_model'].replace('.', '_')}.json"
)

# Sauvegarder les résultats dans un fichier JSON
with open(output_filename, "w") as output_file:
    json.dump(results, output_file, indent=4)

print(f"Les questions et réponses ont été sauvegardées dans {output_filename}.")


Using device: cpu
Les embeddings existent déjà dans ChromaDB.
Les questions et réponses ont été sauvegardées dans comparaisonLLMemb/results_all-MiniLM-L6-v2_gemini-1_5-flash.json.


In [8]:
import os
import json
import google.generativeai as genai
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import torch
import time
import random

# Lire la clé API depuis API.txt
with open("API.txt", "r") as file:
    api_key = file.read().strip()

# Configurer l'API Gemini avec la clé
genai.configure(api_key=api_key)

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Initialisez l'instance de ChromaDB (Vector Store)
db_path = "chroma_db_gem_miniL12"
if not os.path.exists(db_path):
    os.makedirs(db_path)

chroma_client = chromadb.PersistentClient(path=db_path)
collection = chroma_client.get_or_create_collection(name="drug_embeddings")

# Charger le modèle d'embedding
embedding_model_name = 'sentence-transformers/all-MiniLM-L12-v1'
embedding_model = SentenceTransformer(embedding_model_name)
embedding_model.to(device)

def add_embeddings_to_vectorstore(df):
    if collection.count() == 0:  # Vérifie le nombre total de documents dans la collection
        for index, row in df.iterrows():
            embedding = embedding_model.encode(row['combined_text'], device=device).tolist()
            collection.add(
                documents=[row['combined_text']],
                embeddings=[embedding],
                ids=[str(index)]
            )
        print("Embeddings ajoutés à ChromaDB.")
    else:
        print("Les embeddings existent déjà dans ChromaDB.")

# Ajoutez vos données existantes
add_embeddings_to_vectorstore(df_drug)

# Fonction pour utiliser le modèle Gemini
def query_gemini_with_retry(prompt, model_name="gemini-1.5-flash", retries=1):
    for attempt in range(retries):
        try:
            model = genai.GenerativeModel(model_name)
            response = model.generate_content(prompt)
            return response.text.strip()
        except Exception as e:
            print(f"Attempt {attempt + 1} failed: {e}")
            if attempt < retries - 1:
                time.sleep(2 ** attempt + random.random())  # Exponential backoff
            else:
                raise

# Fonction RAG pour répondre aux questions
def rag_pipeline(query, results_number=10, llm_model_name="gemini-1.5-flash"):
    # Générer l'embedding de la requête
    query_embedding = embedding_model.encode(query).tolist()
    
    # Rechercher les contextes pertinents dans ChromaDB
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=results_number
    )
    
    # Construire le contexte pour le modèle LLM
    contexts = results["documents"][0]
    context_text = "\n".join([f"Context {i + 1}: {text}" for i, text in enumerate(contexts)])
    input_prompt = f"""
It's a school project. You are an AI assistant tasked with answering questions using only the information in the provided context. Do not add any extra information or assumptions.

Context:
{context_text}

Question:
{query}

Instructions:
1. Use only the information in the context to answer the question.
2. If the context mentions multiple options, provide a list of those options clearly.
3. If the context does not provide relevant information, state: "The context does not contain enough information to answer this question."
4. Do not include any policy or ethical reasoning in your response.
5. Dont quote the Context and dont mention that you used a context/reviews to answer

Answer:
"""
    # Générer une réponse à l'aide de Gemini
    response = query_gemini_with_retry(input_prompt, model_name=llm_model_name)
    return response

# Charger les questions depuis le fichier JSON
with open("questions.json", "r") as file:
    questions_data = json.load(file)

# Stocker les résultats
results = {
    "meta": {
        "embedding_model": embedding_model_name,
        "llm_model": "gemini-1.5-flash",
        "results_number": 10
    },
    "qa_pairs": []
}

# Limiter à 10 premières questions
questions_subset = questions_data

# Itérer sur les 10 premières questions et collecter les réponses
for question_obj in questions_subset:
    question = question_obj["question"]
    answer = rag_pipeline(question)
    results["qa_pairs"].append({"question": question, "answer": answer})
    time.sleep(10)  # Petite pause entre les appels pour éviter les dépassements de quota

# Définir le répertoire ou fichier cible pour sauvegarder les résultats
output_directory = "comparaisonLLMemb"
os.makedirs(output_directory, exist_ok=True)  # Crée le répertoire s'il n'existe pas

# Générer un nom de fichier dynamique
output_filename = os.path.join(
    output_directory, 
    f"results_{embedding_model_name.split('/')[-1]}_{results['meta']['llm_model'].replace('.', '_')}.json"
)

# Sauvegarder les résultats dans un fichier JSON
with open(output_filename, "w") as output_file:
    json.dump(results, output_file, indent=4)

print(f"Les questions et réponses ont été sauvegardées dans {output_filename}.")


Using device: cpu
Les embeddings existent déjà dans ChromaDB.
Les questions et réponses ont été sauvegardées dans comparaisonLLMemb/results_all-MiniLM-L12-v1_gemini-1_5-flash.json.


In [8]:
import os
import json
import google.generativeai as genai
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import torch
import time
import random

# Lire la clé API depuis API.txt
with open("API.txt", "r") as file:
    api_key = file.read().strip()

# Configurer l'API Gemini avec la clé
genai.configure(api_key=api_key)

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Initialisez l'instance de ChromaDB (Vector Store)
db_path = "chroma_db_gem_mpnet"
if not os.path.exists(db_path):
    os.makedirs(db_path)

chroma_client = chromadb.PersistentClient(path=db_path)
collection = chroma_client.get_or_create_collection(name="drug_embeddings")

# Charger le modèle d'embedding
embedding_model_name = 'sentence-transformers/multi-qa-mpnet-base-dot-v1'
embedding_model = SentenceTransformer(embedding_model_name)
embedding_model.to(device)

def add_embeddings_to_vectorstore(df):
    if collection.count() == 0:  # Vérifie le nombre total de documents dans la collection
        for index, row in df.iterrows():
            embedding = embedding_model.encode(row['combined_text'], device=device).tolist()
            collection.add(
                documents=[row['combined_text']],
                embeddings=[embedding],
                ids=[str(index)]
            )
        print("Embeddings ajoutés à ChromaDB.")
    else:
        print("Les embeddings existent déjà dans ChromaDB.")

# Ajoutez vos données existantes
add_embeddings_to_vectorstore(df_drug)

# Fonction pour utiliser le modèle Gemini
def query_gemini_with_retry(prompt, model_name="gemini-1.5-flash", retries=1):
    for attempt in range(retries):
        try:
            model = genai.GenerativeModel(model_name)
            response = model.generate_content(prompt)
            return response.text.strip()
        except Exception as e:
            print(f"Attempt {attempt + 1} failed: {e}")
            if attempt < retries - 1:
                time.sleep(2 ** attempt + random.random())  # Exponential backoff
            else:
                raise

# Fonction RAG pour répondre aux questions
def rag_pipeline(query, results_number=10, llm_model_name="gemini-1.5-flash"):
    # Générer l'embedding de la requête
    query_embedding = embedding_model.encode(query).tolist()
    
    # Rechercher les contextes pertinents dans ChromaDB
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=results_number
    )
    
    # Construire le contexte pour le modèle LLM
    contexts = results["documents"][0]
    context_text = "\n".join([f"Context {i + 1}: {text}" for i, text in enumerate(contexts)])
    input_prompt = f"""
It's a school project. You are an AI assistant tasked with answering questions using only the information in the provided context. Do not add any extra information or assumptions.

Context:
{context_text}

Question:
{query}

Instructions:
1. Use only the information in the context to answer the question.
2. If the context mentions multiple options, provide a list of those options clearly.
3. If the context does not provide relevant information, state: "The context does not contain enough information to answer this question."
4. Do not include any policy or ethical reasoning in your response.
5. Dont quote the Context and dont mention that you used a context/reviews to answer

Answer:
"""
    # Générer une réponse à l'aide de Gemini
    response = query_gemini_with_retry(input_prompt, model_name=llm_model_name)
    return response

# Charger les questions depuis le fichier JSON
with open("questions.json", "r") as file:
    questions_data = json.load(file)

# Stocker les résultats
results = {
    "meta": {
        "embedding_model": embedding_model_name,
        "llm_model": "gemini-1.5-flash",
        "results_number": 10
    },
    "qa_pairs": []
}

# Limiter à 10 premières questions
questions_subset = questions_data

# Itérer sur les 10 premières questions et collecter les réponses
for question_obj in questions_subset:
    question = question_obj["question"]
    answer = rag_pipeline(question)
    results["qa_pairs"].append({"question": question, "answer": answer})
    time.sleep(10)  # Petite pause entre les appels pour éviter les dépassements de quota

# Définir le répertoire ou fichier cible pour sauvegarder les résultats
output_directory = "comparaisonLLMemb"
os.makedirs(output_directory, exist_ok=True)  # Crée le répertoire s'il n'existe pas

# Générer un nom de fichier dynamique
output_filename = os.path.join(
    output_directory, 
    f"results_{embedding_model_name.split('/')[-1]}_{results['meta']['llm_model'].replace('.', '_')}.json"
)

# Sauvegarder les résultats dans un fichier JSON
with open(output_filename, "w") as output_file:
    json.dump(results, output_file, indent=4)

print(f"Les questions et réponses ont été sauvegardées dans {output_filename}.")


Using device: cpu
Embeddings ajoutés à ChromaDB.
Les questions et réponses ont été sauvegardées dans comparaisonLLMemb/results_multi-qa-mpnet-base-dot-v1_gemini-1_5-flash.json.


# Comparaison

In [12]:
import json
import os

# Fonction pour charger les données des fichiers JSON
def load_json_files(file_paths):
    # Initialisation d'un dictionnaire pour regrouper les réponses par question
    question_data = {}
    
    for idx, file_path in enumerate(file_paths):
        with open(file_path, 'r', encoding='utf-8') as file:
            json_data = json.load(file)
            for qa_pair in json_data.get("qa_pairs", []):
                question = qa_pair.get("question")
                answer = qa_pair.get("answer")
                if question not in question_data:
                    question_data[question] = {}
                # Enregistrer la réponse sous une clé spécifique en fonction de l'ordre des fichiers
                question_data[question][f"answer_{idx+1}"] = answer
    return question_data

# Fonction pour sauvegarder les données combinées dans un seul fichier JSON
def save_to_single_json(question_data, output_dir, output_filename):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Préparer les données sous forme de liste de dictionnaires
    combined_data = [
        {"question": question, **answers}
        for question, answers in question_data.items()
    ]
    
    output_path = os.path.join(output_dir, output_filename)
    with open(output_path, 'w', encoding='utf-8') as file:
        json.dump({"qa_pairs": combined_data}, file, indent=4, ensure_ascii=False)

# Chemins vers vos fichiers JSON
file_paths = [
    "comparaisonLLMemb/results_all-MiniLM-L6-v2_gemini-1_5-flash.json", 
    "comparaisonLLMemb/results_all-MiniLM-L12-v1_gemini-1_5-flash.json", 
    "comparaisonLLMemb/results_multi-qa-mpnet-base-dot-v1_gemini-1_5-flash.json"
]

# Charger les données et regrouper les réponses par question
question_data = load_json_files(file_paths)

# Enregistrer les données combinées dans un seul fichier JSON
output_directory = "comparisonJson"
output_filename = "combined_comparisonJson.json"
save_to_single_json(question_data, output_directory, output_filename)

print(f"Le fichier JSON combiné a été créé dans {os.path.join(output_directory, output_filename)}.")


Le fichier JSON combiné a été créé dans comparisonJson/combined_comparisonJson.json.


In [9]:
from openai import Configuration, OpenAIApi

# Charger la clé API depuis un fichier
def load_api_key(file_path="gptkey.txt"):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read().strip()

# Générer un prompt pour ChatGPT
def generate_prompt(question, answer_1, answer_2, answer_3):
    return (
        f"Here is a question and 3 answers to the question:\n\n"
        f"Question: {question}\n"
        f"Answer 1: {answer_1}\n"
        f"Answer 2: {answer_2}\n"
        f"Answer 3: {answer_3}\n\n"
        f"Imagine you are a simple user asking this question. Rank these answers from best to worst. "
        f"Provide the ranking as a list, for example {{3,1,2}} if the second answer is the best, the third is second, and the first is third."
    )

# Obtenir le classement depuis ChatGPT
def get_ranking_from_chatgpt(question, answer_1, answer_2, answer_3, api_key, organization_id):
    # Configurer OpenAI avec la clé API et l'ID d'organisation
    configuration = Configuration(
        organization=organization_id,
        api_key=api_key
    )
    openai = OpenAIApi(configuration)

    # Préparer le prompt
    prompt = generate_prompt(question, answer_1, answer_2, answer_3)

    try:
        # Appeler l'API OpenAI
        response = openai.createChatCompletion(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "You are an assistant that evaluates answers based on their relevance to a question."},
                {"role": "user", "content": prompt}
            ]
        )
        return response.choices[0].message['content'].strip()
    except Exception as e:
        print(f"Erreur lors de l'appel à l'API OpenAI : {e}")
        return None

# Exemple d'utilisation
if __name__ == "__main__":
    # Charger la clé API
    api_key = load_api_key("gptkey.txt")
    organization_id = "org-Fqi8iCKSJFCBAtfq5T6Sg7qM"  # ID de l'organisation
    
    # Exemple de question et réponses
    question = "What is the capital of France?"
    answer_1 = "Paris"
    answer_2 = "Berlin"
    answer_3 = "Madrid"
    
    # Obtenir le classement
    ranking = get_ranking_from_chatgpt(question, answer_1, answer_2, answer_3, api_key, organization_id)
    print("Classement : ", ranking)


ImportError: cannot import name 'Configuration' from 'openai' (/Users/loshanrasan/anaconda3/lib/python3.11/site-packages/openai/__init__.py)

Erreur lors de l'appel à l'API OpenAI : module 'openai' has no attribute 'Chat'
Classement :  None


In [20]:
import os
import json

def save_questions_to_json_with_global_metadata(json_file, output_dir, questions_per_file=50):
    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)
    
    # Load the JSON data
    with open(json_file, 'r') as file:
        data = json.load(file)["qa_pairs"]
    
    # Break data into chunks of 50 questions
    chunks = [data[i:i + questions_per_file] for i in range(0, len(data), questions_per_file)]
    
    # Loop through chunks and save to JSON files
    for i, chunk in enumerate(chunks):
        output_file = os.path.join(output_dir, f"questions_part_{i+1}.json")
        formatted_chunk = {
            "metadata": {
                "instruction": "Imagine you are a simple user asking these questions. Rank the answers for each question from best to worst. "
                               "Provide the ranking as a list for each question, for example {3,1,2} if the second answer is the best, "
                               "the third is second, and the first is third."
                               "When it says The context does not contain enough information to answer this question. give 4 as un rank even if there is 2 answer with this sentence"
            },
            "questions": []
        }
        for qa in chunk:
            question_data = {
                "question": qa["question"],
                "answers": {
                    "answer_1": qa["answer_1"],
                    "answer_2": qa["answer_2"],
                    "answer_3": qa["answer_3"]
                }
            }
            formatted_chunk["questions"].append(question_data)
        
        # Save the formatted chunk as JSON
        with open(output_file, 'w') as file:
            json.dump(formatted_chunk, file, indent=4)
        
        print(f"Saved JSON file: {output_file}")

# Example usage
save_questions_to_json_with_global_metadata(
    json_file='/Users/loshanrasan/Desktop/IMT MINES ALES/3A/2IA/LLM-Project/DL_Project_RAG/comparisonJson/combined_comparisonJson.json', 
    output_dir='/Users/loshanrasan/Desktop/IMT MINES ALES/3A/2IA/LLM-Project/DL_Project_RAG/output_questions', 
    questions_per_file=50
)


Saved JSON file: /Users/loshanrasan/Desktop/IMT MINES ALES/3A/2IA/LLM-Project/DL_Project_RAG/output_questions/questions_part_1.json
Saved JSON file: /Users/loshanrasan/Desktop/IMT MINES ALES/3A/2IA/LLM-Project/DL_Project_RAG/output_questions/questions_part_2.json
Saved JSON file: /Users/loshanrasan/Desktop/IMT MINES ALES/3A/2IA/LLM-Project/DL_Project_RAG/output_questions/questions_part_3.json
Saved JSON file: /Users/loshanrasan/Desktop/IMT MINES ALES/3A/2IA/LLM-Project/DL_Project_RAG/output_questions/questions_part_4.json
Saved JSON file: /Users/loshanrasan/Desktop/IMT MINES ALES/3A/2IA/LLM-Project/DL_Project_RAG/output_questions/questions_part_5.json
Saved JSON file: /Users/loshanrasan/Desktop/IMT MINES ALES/3A/2IA/LLM-Project/DL_Project_RAG/output_questions/questions_part_6.json
Saved JSON file: /Users/loshanrasan/Desktop/IMT MINES ALES/3A/2IA/LLM-Project/DL_Project_RAG/output_questions/questions_part_7.json
Saved JSON file: /Users/loshanrasan/Desktop/IMT MINES ALES/3A/2IA/LLM-Projec

# Analyse

In [11]:
import pandas as pd
import json

# Chemin du fichier JSON
fichier_json = "/Users/loshanrasan/Desktop/IMT MINES ALES/3A/2IA/LLM-Project/DL_Project_RAG/reponses.json"

# Chargement des données JSON
with open(fichier_json, 'r') as f:
    donnees = json.load(f)

# Création du DataFrame
colonnes = ["L6-V2 - Embedding", "L12-V1 - Embedding", "Mpnet - Embedding"]
df = pd.DataFrame(donnees["rankings"], columns=colonnes)




In [12]:
df

,L6-V2 - Embedding,L12-V1 - Embedding,Mpnet - Embedding
0,3,2,1
1,2,3,1
2,2,1,3
3,1,3,2
4,3,2,1
...,...,...,...
395,4,4,4
396,2,1,3
397,1,2,3
398,1,3,2


In [13]:

statistics = {
    "Average Rank": df.mean(),
    "Number of First Places": (df == 1).sum(),
    "Percentage of First Places (%)": ((df == 1).sum() / len(df) * 100),
    "Number of No Responses": (df == 4).sum(),
    "Percentage of No Responses (%)": (df == 4).sum()/len(df)*100
}

# Convert the statistics into a DataFrame
stats_df = pd.DataFrame(statistics)

stats_df

,Average Rank,Number of First Places,Percentage of First Places (%),Number of No Responses,Percentage of No Responses (%)
L6-V2 - Embedding,2.2100,152,38.00,74,18.5
L12-V1 - Embedding,2.3275,93,23.25,66,16.5
Mpnet - Embedding,2.6450,69,17.25,96,24.0


In [19]:
# Remove rows where all three columns have the value 4
filtered_df = df[~(df == 4).all(axis=1)]
print(len(filtered_df))
# Calculate statistics on the filtered DataFrame
statistics = {
    "Average Rank": filtered_df.mean(),
    "Number of First Places": (filtered_df == 1).sum(),
    "Percentage of First Places (%)": ((filtered_df == 1).sum() / len(filtered_df) * 100),
    "Number of No Responses": (filtered_df == 4).sum(),
    "Percentage of No Responses (%)": (filtered_df == 4).sum() / len(filtered_df) * 100
}

# Convert the statistics into a DataFrame
stats_df = pd.DataFrame(statistics)

stats_df


340


,Average Rank,Number of First Places,Percentage of First Places (%),Number of No Responses,Percentage of No Responses (%)
L6-V2 - Embedding,1.894118,152,44.705882,14,4.117647
L12-V1 - Embedding,2.032353,93,27.352941,6,1.764706
Mpnet - Embedding,2.405882,69,20.294118,36,10.588235


In [20]:
# Remove rows where any column has the value 4
filtered_df = df[~(df == 4).any(axis=1)]

print(len(filtered_df))  # Print the number of remaining rows

# Calculate statistics on the filtered DataFrame
statistics = {
    "Average Rank": filtered_df.mean(),
    "Number of First Places": (filtered_df == 1).sum(),
    "Percentage of First Places (%)": ((filtered_df == 1).sum() / len(filtered_df) * 100),
    "Number of No Responses": (filtered_df == 4).sum(),
    "Percentage of No Responses (%)": (filtered_df == 4).sum() / len(filtered_df) * 100
}

# Convert the statistics into a DataFrame
stats_df = pd.DataFrame(statistics)

stats_df


302


,Average Rank,Number of First Places,Percentage of First Places (%),Number of No Responses,Percentage of No Responses (%)
L6-V2 - Embedding,1.801325,141,46.688742,0,0.0
L12-V1 - Embedding,1.983444,92,30.463576,0,0.0
Mpnet - Embedding,2.215232,69,22.847682,0,0.0


# ELECTRE TRI

In [10]:
import numpy as np
import pandas as pd


# --- Étapes de ELECTRE TRI ---

# 1. Initialisation : Pondération uniforme
weights = np.ones(df.shape[0]) / df.shape[0]  # Pondérations égales

# 2. Matrice de concordance
def concordance_matrix(df):
    num_models = df.shape[1]
    concordance = np.zeros((num_models, num_models))
    for i in range(num_models):
        for j in range(num_models):
            if i != j:
                concordance[i, j] = np.sum(
                    weights[df.iloc[:, i] <= df.iloc[:, j]]
                )
    return concordance

# 3. Matrice de discordance
def discordance_matrix(df):
    num_models = df.shape[1]
    discordance = np.zeros((num_models, num_models))
    for i in range(num_models):
        for j in range(num_models):
            if i != j:
                discordance[i, j] = np.max(
                    df.iloc[:, i] - df.iloc[:, j]
                ) / np.max(df.values)
    return discordance

# 4. Seuils de concordance et discordance
concordance_threshold = 0.5
discordance_threshold = 0.5

# 5. Matrice d’agrégation
def aggregated_dominance(concordance, discordance, c_thresh, d_thresh):
    dominance = (concordance >= c_thresh) & (discordance <= d_thresh)
    return dominance.astype(int)

# Calcul des matrices
concordance = concordance_matrix(df)
discordance = discordance_matrix(df)
dominance = aggregated_dominance(concordance, discordance, concordance_threshold, discordance_threshold)

# Résultats
print("Matrice de Concordance:")
print(concordance)

print("\nMatrice de Discordance:")
print(discordance)

print("\nMatrice de Dominance (Agrégée):")
print(dominance)

# Interprétation : Classement des modèles
scores = dominance.sum(axis=1)
ranking = np.argsort(-scores)  # Classement en ordre décroissant des scores
print("\nClassement des modèles (du meilleur au pire):")
for rank, model_index in enumerate(ranking, 1):
    print(f"Modèle {df.columns[model_index]} : Rang {rank}")


Matrice de Concordance:
[[0.     0.6375 0.7025]
 [0.515  0.     0.6975]
 [0.4775 0.4675 0.    ]]

Matrice de Discordance:
[[0.   0.5  0.5 ]
 [0.75 0.   0.5 ]
 [0.75 0.75 0.  ]]

Matrice de Dominance (Agrégée):
[[0 1 1]
 [0 0 1]
 [0 0 0]]

Classement des modèles (du meilleur au pire):
Modèle L6-V2 - Embedding : Rang 1
Modèle L12-V1 - Embedding : Rang 2
Modèle Mpnet - Embedding : Rang 3
